Công thức Newton Cotez n:
$$
I = \int_{x_0}^{x_n} f(x) dx = h\left(a_0y_0 + a_1y_1 + ... + a_ny_n\right)
$$
Chia bảng dữ liệu thành các đoạn con, mỗi đoạn n+1 phần tử
$$
[x_0, x_n], \quad [x_n, x_{2n}], \quad, [x_{2n}, x_{3n}], ...
$$
Tính tích phân trên các đoạn này, sau đó tính tổng các tích phân vừa tính được để ra tích phân trên toàn miền
$$
I = \int_a^b f(x)dx = I_1 + I_2 + ...
$$

In [23]:
import numpy as np
import math
import sys
import matplotlib.pyplot as plt

In [24]:
def Horner_Nhan(a, c):
    ans = np.zeros(len(a)+1)
    P_n = np.zeros(len(a)+1)
    P_n[:-1] = a
    ans[0] = P_n[0]
    for i in range(1, len(ans)):
        ans[i] = P_n[i] - c*P_n[i-1]
    return ans

In [25]:
def Horner_Chia(a, c):
    ans = np.zeros(len(a))
    ans[0] = a[0]
    for i in range(1,len(a)):
        ans[i] = c*ans[i-1] + a[i]
    P_c = ans[-1]
    P_n = ans[:-1]
    return P_n, P_c

Gói con tính hệ số của Newton Cotez và ước lượng sai số

Input: 

    n

Output:

    Hệ số cho Newton Cotez n, công thức sai số toàn cục

In [26]:
def NewtonCotez(n):
    omega = np.array([1])
    HeSo = np.zeros(n+1)
    v = np.zeros((n+1, 1))
    P = np.eye(n+1)
    for i in range(n+1):
        omega = Horner_Nhan(omega, i)
    for i in range(n+1):
        P[i], _ = Horner_Chia(omega, i)
    v[-1] = np.array([n])
    for i in range(n+1):
        P[:, i] /= n+1-i
        if i>0:
            v[n-i] = v[n-i+1]*n
    for i in range(n+1):
        temp = 1
        for j in range(n+1):
            if j!=i:
                temp *= i-j
        P[i] /= temp
    HeSo = np.dot(P, v)

    #Khai trien Taylor de uoc luong sai so

    CapChinhXac = 1
    Y = np.zeros(n+1)
    I = n**2/2
    for i in range(n+1):
        Y[i] = i
    M = I-np.dot(Y, HeSo)
    while abs(M[0]) < 1e-10:
        CapChinhXac += 1
        I *= n/(CapChinhXac+1)
        for i in range(n+1):
            Y[i] *= i/(CapChinhXac)
        M = I-np.dot(Y, HeSo)
    return HeSo, CapChinhXac, abs(M[0])


Tính hệ số và ước lượng sai số của Newton Cotez n

In [27]:
HeSo, CapChinhXac, M = NewtonCotez(6)
print("He so Newton Cotez:")
print(HeSo)
print("Sai so: |I - I*| <=", abs(M), "*M_", CapChinhXac, "*h^", CapChinhXac)


He so Newton Cotez:
[[0.29285714]
 [1.54285714]
 [0.19285714]
 [1.94285714]
 [0.19285714]
 [1.54285714]
 [0.29285714]]
Sai so: |I - I*| <= 0.006428571428354957 *M_ 8 *h^ 8


Gói con tính tích phân khi biết hàm dưới dạng bảng dữ liệu

Input:
    Bộ điểm $(x_i, y_i)$ (mỗi 1 điểm ghi trên 1 dòng, $x_i$ xếp theo thứ tự tăng dần)

Output: 
    Tích phân trên đoạn $[x_0, x_N]$

Ý tưởng:

Tạo một bảng dữ liệu với bước lưới h, tính tích phân dựa trên bảng này

Chia đôi h và tạo bảng mới, tiếp tục như vậy đến khi đạt được sai số mong muốn (đánh giá bằng công thức lưới phủ)



In [28]:
def TinhTichPhan(x, y, n):
    h = x[1] - x[0]
    ans = 0
    N = len(x)
    if (N-1)%n != 0:
        print(N-1, n)
        print("Số đoạn chia không chia hết cho ", n)
        return
    else:
        HeSoNC, _, _ = NewtonCotez(n)
        i = 0
        while i+n <= N-1:
            u = np.zeros((1, n+1))
            for j in range(n+1):
                u[0, j] = y[i+j]
            temp = np.dot(u, HeSoNC)
            ans += temp[0]*h
            i += n
        return ans

Tính tích phân của một hàm số cụ thể 

Input: 
    
    Hàm số f(x), đoạn lấy tích phân [A, B], 

    sai số Epsilon,
    
    N (Công thức Newton Cotez muốn sử dụng)

In [29]:
def f(x):
    return 1/(1+x*x)
A = 0
B = 1
Epsilon = 0.5*1e-7
N = 1  # Công thức Newton Cotez muốn sử dụng

In [30]:
def ChenDiemNoiSuy(x, y):
    n = len(x)
    i = 0
    while i<n-1:
        t = (x[i]+x[i+1])/2
        x = np.insert(x, i+1, t)
        y = np.insert(y, i+1, f(t))
        i += 2
        n += 1
    return x, y


In [31]:
def TichPhanHamChoTruoc(a, b, epsilon, n):
    x = np.zeros(n+1)
    y = np.zeros(n+1)
    count = 0
    I_h = 0
    I_h2 = 0
    x[0] = a 
    x[-1] = b 
    y[0] = f(x[0])
    y[-1] = f(x[-1])
    h = abs(b-a)/n
    for i in range(1, n):
        x[i] = x[0] + i*h
        y[i] = f(x[i])
    _, CapChinhXac, _ = NewtonCotez(n)
    I_h2 = TinhTichPhan(x, y, n)
    print("Sai số: |I - I*| <= 1/", 2**CapChinhXac -1, "*|I_h - I_{2h}| <= Epsilon")
    print("interation", count, "\t h =", h, "\tI_h =", I_h2," \tI_2h =", I_h, "\t\tDelta I_h = ", abs(I_h - I_h2))
    while abs(I_h - I_h2) > (2**CapChinhXac-1)*Epsilon:
        I_h = I_h2
        x, y = ChenDiemNoiSuy(x, y)
        I_h2 = TinhTichPhan(x, y, n)
        count += 1
        h /= 2
        print("interation", count, "\t h =", h, "\tI_h =", I_h2," \tI_2h =", I_h, "\tDelta I_h = ", abs(I_h - I_h2))
    #return I_h2

TichPhanHamChoTruoc(A, B, Epsilon, N)


Sai số: |I - I*| <= 1/ 3 *|I_h - I_{2h}| <= Epsilon
interation 0 	 h = 1.0 	I_h = [0.75]  	I_2h = 0 		Delta I_h =  [0.75]
interation 1 	 h = 0.5 	I_h = [0.775]  	I_2h = [0.75] 	Delta I_h =  [0.025]
interation 2 	 h = 0.25 	I_h = [0.78279412]  	I_2h = [0.775] 	Delta I_h =  [0.00779412]
interation 3 	 h = 0.125 	I_h = [0.78474712]  	I_2h = [0.78279412] 	Delta I_h =  [0.00195301]
interation 4 	 h = 0.0625 	I_h = [0.7852354]  	I_2h = [0.78474712] 	Delta I_h =  [0.00048828]
interation 5 	 h = 0.03125 	I_h = [0.78535747]  	I_2h = [0.7852354] 	Delta I_h =  [0.00012207]
interation 6 	 h = 0.015625 	I_h = [0.78538799]  	I_2h = [0.78535747] 	Delta I_h =  [3.05175777e-05]
interation 7 	 h = 0.0078125 	I_h = [0.78539562]  	I_2h = [0.78538799] 	Delta I_h =  [7.62939452e-06]
interation 8 	 h = 0.00390625 	I_h = [0.78539753]  	I_2h = [0.78539562] 	Delta I_h =  [1.90734863e-06]
interation 9 	 h = 0.001953125 	I_h = [0.785398]  	I_2h = [0.78539753] 	Delta I_h =  [4.76837159e-07]
interation 10 	 h = 0.0

Tính tích phân của một hàm số khi biết các điểm rời rạc (Bảng dữ liệu)

Input: 

Bộ điểm $(x_i, y_i)$ được ghi vào file Data.txt (mỗi điểm ghi trên 1 dòng, $x_i$ xếp theo thứ tự tăng dần)

n (Công thức Newton Cotez muốn sử dụng)


Output: 

Tích phân cần tính, sai số (nếu đánh giá đc)

In [32]:
n = 2

sys.stdin = open('Data.txt', 'r')
first_line = sys.stdin.readline()
sys.stdin.close()
sys.stdin = open('Data.txt', 'r')
temp = first_line
temp = temp.split()
X = []
Y = []
if len(temp) == 2:
    for line in sys.stdin:
        diem_noi_suy = np.array([float(toa_do) for toa_do in line.split()])
        X.append(diem_noi_suy[0])
        Y.append(diem_noi_suy[1])
    X = np.array(X)
    Y = np.array(Y)
else:
    x_line = sys.stdin.readline()
    X = np.array([float(x_i) for x_i in x_line.split()])
    y_line = sys.stdin.readline()
    Y = np.array([float(y_i) for y_i in y_line.split()])

sys.stdin.close()

In [33]:

lenX = len(X)
I_h = TinhTichPhan(X, Y, n)
print("I_h = ", I_h)
if ((lenX-1)/2)%n == 0:
    X_2h = np.array([])
    Y_2h = np.array([])
    k = 0
    while k <= lenX-1:
        X_2h = np.append(X_2h, X[k])
        Y_2h = np.append(Y_2h, Y[k])
        k += 2
    I_2h = TinhTichPhan(X_2h, Y_2h, n)
    print("I_2h = ", I_2h)
    print("Sai so: |I_h - I| <= 1 /",2**n-1 ," * |I_h - I_2h|  = ", abs(I_h-I_2h)/(2**n -1))



I_h =  [-7.08643278]
I_2h =  [-7.08667027]
Sai so: |I_h - I| <= 1 / 3  * |I_h - I_2h|  =  [7.91633333e-05]
